In [13]:
%pylab inline

import tensorflow.keras as keras
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


In [4]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.text', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [40]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [41]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [68]:
encoded = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [125]:
one_hots = np.zeros(shape=(1,encoded.shape[1],max_id))

In [ ]:
one_hots[0,:,encoded[0]]

In [116]:
tf.one_hot(encoded[0][0], depth=max_id)

<tf.Tensor: shape=(39,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.], dtype=float32)>

In [113]:
np.map(lambda char: tf.one_hot(char, depth=max_id), encoded)

AttributeError: module 'numpy' has no attribute 'map'

In [88]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [89]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [90]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [91]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:,1:]))

In [92]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [100]:
dataset = dataset.prefetch(1)

In [101]:
dataset

<PrefetchDataset shapes: ((None, None, 1115394, 39), (None, None, 1115394)), types: (tf.float32, tf.int64)>

In [63]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=(None, max_id), dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_16 (GRU)                 (None, None, 128)         64896     
_________________________________________________________________
gru_17 (GRU)                 (None, None, 128)         99072     
_________________________________________________________________
time_distributed_8 (TimeDist (None, None, 39)          5031      
Total params: 168,999
Trainable params: 168,999
Non-trainable params: 0
_________________________________________________________________


In [64]:
history = model.fit(dataset, epochs=20)

ValueError: in converted code:

    /Users/isaacdrachman/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py:677 map_fn
        batch_size=None)
    /Users/isaacdrachman/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py:2410 _standardize_tensors
        exception_prefix='input')
    /Users/isaacdrachman/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_utils.py:573 standardize_input_data
        'with shape ' + str(data_shape))

    ValueError: Error when checking input: expected gru_16_input to have 3 dimensions, but got array with shape (None, None, 1115394, 39)


In [65]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

def predict(feed):
    X_new = preprocess([feed])
    y_proba = model.predict(X_new)[0,-1:,:]
    rescaled = tf.math.log(y_proba)
    char_id = tf.random.categorical(rescaled, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]